In [207]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from torch.nn.functional import softmax
from torch.utils.data import DataLoader
import pandas as pd

In [3]:
model_name = "AbdulmohsenA/Faseeh"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

# Load your full translation dataset
dataset = load_dataset("Abdulmohsena/Classic-Arabic-English-Language-Pairs")['train']

In [232]:
def calculate_entropy(outputs):
    
    logits = torch.stack(outputs['scores']).permute(1, 0, 2)
    probs = softmax(logits, dim=-1)
    
    padding_mask = (outputs['sequences'][:, 1:] != 1).float() # To cancel padding tokens
    
    token_entropy = (-torch.sum(probs * torch.log(probs + 1e-8), dim=-1)) * padding_mask
    
    sentence_entropy = token_entropy.sum(dim=-1) / padding_mask.sum(dim=-1)  # Get the mean from valid tokens
    
    return sentence_entropy

In [239]:
dataset_uncertainties = pd.DataFrame(columns=['source', 'target', 'pred', 'entropy'])

for batch in dataloader:
    inputs = tokenizer(batch['source'], return_tensors='pt', padding=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)
    
    entropy = calculate_entropy(outputs)

    pred = tokenizer.batch_decode(outputs['sequences'], skip_special_tokens=True)

    new_data = pd.DataFrame({
        'source': batch['source'],
        'target': batch['target'],
        'pred': pred,
        'entropy': entropy
    })
    
    dataset_uncertainties = pd.concat([dataset_uncertainties, new_data], ignore_index=True)

{'source': [' this gradualism is what caught the attention of the brethren of purity. after they decided that souls vary in love between a lustful soul, an animal soul, and a royal soul, and that in the nature of souls is the love of eternal survival, they spoke:', 'fie upon you, and upon that which you worship besides allah! have you then no sense', 'this is better for you than that begging should come as a spot on your face on the day of judgment. begging is right only for three people: for one who is in grinding poverty, or for one who is in severe debt, or for a painful compensation for killing.', 'it is allah, except whom there is no god the knowing of all – the hidden and the evident he only is the most gracious, the most merciful.'], 'target': ['وهذا التدرج هو الذي لفت أنظار إخوان الصفا، فبعد أن قرروا أن النفوس تتفاوت في المحبة بين نفس شهوانية وأخرى حيوانية وثالثة ملكية وأن في جبلة النفوس محبة البقاء السرمدي تحدثوا', 'أف لكم ولما تعبدون من دون الله أفلا تعقلون', 'هذا خير لك من أ

C:\Users\user\AppData\Local\Temp\ipykernel_11044\3115130936.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_uncertainties = pd.concat([dataset_uncertainties, new_data], ignore_index=True)


{'source': ['did you see him who gives the lie to the reward and punishment of the hereafter?', 'i saw the prophet being served with soup and containing gourd and cured meat, and i saw him pickingand eating the pieces of gourd.', 'this is nothing but the word of mortal man!', 'there is a chapter in the quran of thirty verses which will intercede for its companion until he is forgiven. it is: blessed is he in whose hand is the kingdom. '], 'target': ['أرأيت الذي يكذب بالدين', 'رأيت النبي أتي بمرقة فيها دباء وقديد، فرأيته يتتبع الدباء يأكلها.', 'إن هذا إلا قول البشر', 'سورة من القرآن ثلاثون آية تشفع لصاحبها حتى يغفر له تبارك الذي بيده الملك']}


In [240]:
dataset_uncertainties

,source,target,pred,entropy
0,this gradualism is what caught the attention ...,وهذا التدرج هو الذي لفت أنظار إخوان الصفا، فبع...,وهذه التدرجية هي التي استسرقت همة إخوان الصفا....,0.699131
1,"fie upon you, and upon that which you worship ...",أف لكم ولما تعبدون من دون الله أفلا تعقلون,أف لكم ولما تعبدون من دون الله أفلا تعقلون,0.000812
2,this is better for you than that begging shoul...,هذا خير لك من أن تجيء المسألة نكتة في وجهك يوم...,هذا خير لك من أن يأتيك المسألة رقعة في وجهك يو...,1.275959
3,"it is allah, except whom there is no god the k...",هو الله الذي لا إله إلا هو عالم الغيب والشهادة...,هو الله الذي لا إله إلا هو عالم الغيب والشهادة...,0.002247
4,did you see him who gives the lie to the rewar...,أرأيت الذي يكذب بالدين,أرأيت الذي يكذب بالدين,0.109136
5,i saw the prophet being served with soup and c...,رأيت النبي أتي بمرقة فيها دباء وقديد، فرأيته ي...,رأيت النبي يقدم له حساء فيه الدباء واللحم الصا...,1.685970
6,this is nothing but the word of mortal man!,إن هذا إلا قول البشر,إن هذا إلا قول البشر,0.072779
7,there is a chapter in the quran of thirty vers...,سورة من القرآن ثلاثون آية تشفع لصاحبها حتى يغف...,سورة من القرآن ثلاثون آية تشفع لصاحبها حتى يغف...,0.678057


In [71]:
import pandas as pd

df = pd.read_csv("./uncertains.csv", index_col='Unnamed: 0')

In [72]:
df.sample(1).values

array([['and say thou: all praise unto allah! anon he shall shew you his signs, so that ye will recognize them. and thy lord is not negligent of that which ye work.',
        'وقل الحمد لله سيريكم آياته فتعرفونها وما ربك بغافل عما تعملون',
        'وقل الحمد لله سيريكم آياته فتعرفونها وما ربك بغافل عما تعملون',
        0.0014377779]], dtype=object)

In [73]:
threshold = 30000

df = df.sort_values(by="entropy", ascending=False).iloc[:threshold]
df = df[~df['source'].str.contains(r'reported', case=False, na=False)]
df.dropna(inplace=True)

In [74]:
df.reset_index(drop=True, inplace=True)

In [75]:
df

,source,target,pred,entropy
0,two more versions of the tradition narrated by...,عن النبي بمعنى حديث أبي حازم عن بعجة وقال في ش...,قال رسول الله قال رسول الله قال رسول الله قال ...,3.614428
1,the messenger of allah rinsed his mouth and sn...,أن رسول الله مضمض واستنشق من غرفة واحدة,أن رسول الله مر على فيه من ماء واحد فتناهاه,3.329385
2,she unveiled the hyacinths of the ugly ones l...,أماطت لثاما عن أقاحي الدمائث بمثل أساريع الحقو...,كشفت عن حياصن البقيان كعثا السريع الشوكا بالوديان,3.208522
3,none amongst the created beings who had been l...,عن النبي أنه قال ذلك قبل موته بشهر أو نحو ذلك ...,قال لم يكن في ذلك العالم أحد قال عبد الرحمن قا...,3.194528
4,to hold the hands by the hands in prayer belo...,قال كان رسول الله يصلي بنا فيقرأ في الظهر والع...,أن يمنع من الصلاة أن يمنع من السرة,3.121780
...,...,...,...,...
29321,i attended the council of abu bakr bin duraid...,حضرت ببغداد مجلس أبي بكر بن دريد ، وأبو نصر هذ...,حضرت مجلس أبي بكر بن دريد ببغداد، وأبو نصر يقر...,0.541948
29322,professor raif khoury wrote a study about it ...,وكتب عنها الأستاذ رئيف خوري دراسة بعنوان حول ك...,وقد كتب الأستاذ رئيف خوري دراسة في ذلك بعنوان ...,0.541921
29323,the boat belonged to some destitute people who...,أما السفينة فكانت لمساكين يعملون في البحر فأرد...,أما السفينة فكانت لمساكين يعملون في البحر وكان...,0.541903
29324,91 39 alawite is proud of himself,٩١ ٣٩ علوي يفتخر بنفسه,٩١ ٣٩ العلوي يفتخر بنفسه,0.541819


In [76]:
from datasets import Dataset

df = df.drop(columns=['pred', 'entropy'])

In [77]:
df

,source,target
0,two more versions of the tradition narrated by...,عن النبي بمعنى حديث أبي حازم عن بعجة وقال في ش...
1,the messenger of allah rinsed his mouth and sn...,أن رسول الله مضمض واستنشق من غرفة واحدة
2,she unveiled the hyacinths of the ugly ones l...,أماطت لثاما عن أقاحي الدمائث بمثل أساريع الحقو...
3,none amongst the created beings who had been l...,عن النبي أنه قال ذلك قبل موته بشهر أو نحو ذلك ...
4,to hold the hands by the hands in prayer belo...,قال كان رسول الله يصلي بنا فيقرأ في الظهر والع...
...,...,...
29321,i attended the council of abu bakr bin duraid...,حضرت ببغداد مجلس أبي بكر بن دريد ، وأبو نصر هذ...
29322,professor raif khoury wrote a study about it ...,وكتب عنها الأستاذ رئيف خوري دراسة بعنوان حول ك...
29323,the boat belonged to some destitute people who...,أما السفينة فكانت لمساكين يعملون في البحر فأرد...
29324,91 39 alawite is proud of himself,٩١ ٣٩ علوي يفتخر بنفسه


In [78]:
dataset = Dataset.from_pandas(df)

In [79]:
dataset.push_to_hub("AbdulmohsenA/starter")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/312 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Abdulmohsena/starter/commit/544019ebca1aac3d3e5662a9a024e2296f2f90d1', commit_message='Upload dataset', commit_description='', oid='544019ebca1aac3d3e5662a9a024e2296f2f90d1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Abdulmohsena/starter', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Abdulmohsena/starter'), pr_revision=None, pr_num=None)